In [116]:
import requests
import itertools
import re
import random
import time
from lxml import html

In [163]:
langs = [
	"eng", "deu", "swe", "nld", # germanski
	"ita", "por", "spa", "fra", # romanski
	"slv", "hrv", "srp", "rus", # slovanski
	"hin", "ell", "arb", "hun", # ostalo
	"zho", "fin", "jpn", "kor"
]

data = {}

In [178]:
for lang in langs: # poišči vse verzije biblije za dan jezik
	versions_res = requests.get("https://bible.youversionapi.com/3.2/versions.json", params={"language_tag": lang, "type": "all"}).json()
	ok = False
	print(lang)
	if data[lang] and len(data[lang]['books']) >= 66: continue # preskoči jezike ki jih že imaš
	time.sleep(1)
	for version in versions_res["response"]["data"]["versions"]: # poišči tako verzijo, ki ima vse* knjige
		id = version['id']
		books_res = requests.get(f"https://bible.youversionapi.com/3.2/version.json?id={id}").json()
		books = books_res["response"]["data"]["books"]
		# print(len(books))
		if len(books) >= 66:
			data[lang] = {"id": id, "books": books}
			ok = True
			break
	if not ok: print(f"'{lang}' missing books!")

eng
deu
swe
nld
ita
por
spa
fra
slv
hrv
srp
rus
hin
ell
arb
hun
zho
fin
jpn
kor


In [204]:
# izberi 5 knjig
books_intersection = set(map(lambda b: b["usfm"], data[langs[0]]['books']))
for lang in langs:
	books = set(map(lambda b: b["usfm"], data[langs[0]]['books']))
	books_intersection = books_intersection.intersection(books)

selected_books = random.sample(books_intersection, k=5)
print(selected_books)

['ZEC', 'LUK', 'ISA', 'NUM', 'JUD']


In [201]:
def clean(content):
	dom = html.fromstring(re.sub("</", " </", content))
	for el in itertools.chain(dom.find_class("label"), dom.find_class("heading")):
		el.getparent().remove(el)
	return re.sub('\s+',' ',dom.text_content())

In [208]:
for lang in data:
	if not data[lang]['books_text']:
		data[lang]['books_text'] = {}
	for book in data[lang]["books"]:
		if book["usfm"] in selected_books:
			print(lang, book["usfm"])
			if data[lang]['books_text'].get(book["usfm"], False): continue
			text = ""
			for chapter in book["chapters"]:
				time.sleep(0.1)
				res = requests.get(f"https://bible.youversionapi.com/3.2/chapter.json?id={data[lang]['id']}&reference={chapter['usfm']}").json()
				print(f"https://bible.youversionapi.com/3.2/chapter.json?id={data[lang]['id']}&reference={chapter['usfm']}")
				text += clean(res["response"]["data"]["content"])
				if len(text) > 1000: break
			data[lang]['books_text'][book["usfm"]] = text

ell NUM
https://bible.youversionapi.com/3.2/chapter.json?id=921&reference=NUM.1


KeyError: 'content'

In [211]:
list(map(lambda b: b['usfm'], data["ell"]["books"])).index("NUM")

3

In [218]:
data["slv"]["id"]

2318

In [217]:
data["ell"]["books"][3]

{'text': True,
 'canon': 'ot',
 'chapters': [{'toc': True, 'usfm': 'NUM.1', 'human': '1', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.2', 'human': '2', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.3', 'human': '3', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.4', 'human': '4', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.5', 'human': '5', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.6', 'human': '6', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.7', 'human': '7', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.8', 'human': '8', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.9', 'human': '9', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.10', 'human': '10', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.11', 'human': '11', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.12', 'human': '12', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.13', 'human': '13', 'canonical': True},
  {'toc': True, 'usfm': 'NUM.14', 'human': '14', 'canonical': True},
  {'toc':

In [86]:
# get books
res = requests.get("https://bible.youversionapi.com/3.2/chapter.json?id=1&reference=JHN.1").json()

In [202]:
clean(res["response"]["data"]["content"])

" In the beginning was the Word, and the Word was with God, and the Word was God. The same was in the beginning with God. All things were made by him; and without him was not any thing made that was made. In him was life; and the life was the light of men. And the light shineth in darkness; and the darkness comprehended it not. There was a man sent from God, whose name was John. The same came for a witness, to bear witness of the Light, that all men through him might believe. He was not that Light, but was sent to bear witness of that Light. That was the true Light, which lighteth every man that cometh into the world. He was in the world, and the world was made by him, and the world knew him not. He came unto his own, and his own received him not. But as many as received him, to them gave he power to become the sons of God, even to them that believe on his name: which were born, not of blood, nor of the will of the flesh, nor of the will of man, but of God. And the Word was made flesh,

In [2]:
"to_je_test".split("_")

['to', 'je', 'test']